In [ ]:
import numpy as np
import gym
import k3d
from ratelimiter import RateLimiter
from k3d.platonic import Cube
from time import time

rate_limiter = RateLimiter(max_calls=4, period=1)

env = gym.make('CartPole-v0')
observation = env.reset()

plot = k3d.plot(grid_auto_fit=False, camera_auto_fit=False, grid=(-1,-1,-1,1,1,1))

joint_positions = np.array([observation[0], 0, 0], dtype=np.float32)
pole_positions = joint_positions + np.array([np.sin(observation[2]), 0, np.cos(observation[2])], dtype=np.float32)

cart = Cube(origin=joint_positions, size=0.1).mesh
cart.scaling = [1, 0.5, 1]

joint = k3d.points(np.mean(cart.vertices[[0,2,4,6]], axis=0), point_size=0.03, color=0xff00, shader='mesh')
pole = k3d.line(vertices=np.array([joint.positions, pole_positions]), shader='mesh', color=0xff0000)
box = cart.vertices
mass = k3d.points(pole_positions, point_size=0.03, color=0xff0000, shader='mesh')

plot += pole + cart + joint + mass

plot.display()

In [ ]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        with rate_limiter:
            joint_positions = np.array([observation[0], 0, 0], dtype=np.float32)
            pole_positions = joint_positions + np.array([np.sin(observation[2]), 0, np.cos(observation[2])], dtype=np.float32)

            cart.vertices = box + joint_positions
            joint.positions = np.mean(cart.vertices[[0,2,4,6]], axis=0)
            pole.vertices = [joint.positions, pole_positions]
            mass.positions = pole_positions
            
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break

In [ ]:
plot.display()

In [ ]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        
        joint_positions = np.array([observation[0], 0, 0], dtype=np.float32)
        pole_positions = joint_positions + np.array([np.sin(observation[2]), 0, np.cos(observation[2])], dtype=np.float32)
        
        with rate_limiter:
            cart.vertices = box + joint_positions
            joint.positions = np.mean(cart.vertices[[0,2,4,6]], axis=0)
            pole.vertices = [joint.positions, pole_positions]
            mass.positions = pole_positions

        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        
        if done:
            break

In [ ]:
max_calls, period = 4, 1
call_time = period/max_calls

for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        
        joint_positions = np.array([observation[0], 0, 0], dtype=np.float32)
        pole_positions = joint_positions + np.array([np.sin(observation[2]), 0, np.cos(observation[2])], dtype=np.float32)
        time_stamp2 = time()
        
        if t>0:
            d = time_stamp2 - time_stamp1
            if d < call_time:
                cart.vertices = box + joint_positions
                joint.positions = np.mean(cart.vertices[[0,2,4,6]], axis=0)
                pole.vertices = [joint.positions, pole_positions]
                mass.positions = pole_positions
        if t==0:
            cart.vertices = box + joint_positions
            joint.positions = np.mean(cart.vertices[[0,2,4,6]], axis=0)
            pole.vertices = [joint.positions, pole_positions]
            mass.positions = pole_positions

        
        time_stamp1 = time()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        
        if done:
            break